# AWS Marketplace Product Usage Demonstration - Synthetic Data Generator (Tabular) Model Package

## Synthetic Data Generator - Tabular

This sample notebook demonstrates the usage of the Tabular Synthetic Data Generator.

This solution is a GAN based model to generate synthetic tabular data from a sample representative data.

This sample notebook shows you how to deploy Tabular Synthetic Data Generator  using Amazon SageMaker.

**Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

#### Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  
    2. or your AWS account has a subscription to Car Mileage Predictor. If so, skip step: [Subscribe to the model package](#1.-Subscribe-to-the-model-package)

#### Contents:
1. [Subscribe to the model package](# Subscribe-to-the-model-package)
2. [Set up the environment](# Setup the environment)
3. [Create the session](# Create the session)
4. [Create Model](# Create model)
5. [Create an endpoint and perform real-time inference](#-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Visualize output](#D.-Visualize-output)
   5. [Delete the endpoint](#E.-Delete-the-endpoint)
6. [Perform batch inference](# Perform-batch-inference)
    1. [Configure the input S3 bucket folder](# A. -Configure the input S3 bucket folder)
    2. [Deploy the model](# B. -Deploy the model)
    3. [Download the file from output S3 bucket folder](# C. -Download the file from output S3 bucket folder)
    4. [Visualize data](# D. -Visualize data)
7. [Clean-up](# Clean-up)
    1. [Delete the model](#A.-Delete-the-model)
    2. [Unsubscribe to the listing (optional)](#B.-Unsubscribe-to-the-listing-(optional))
    

#### Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to the model package

To subscribe to the model package:
1. Open the model package listing page Car Mileage Predictor. 
1. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
1. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms. 
1. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.

## 2. Set up the environment

In [1]:
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
from sagemaker.serializers import CSVSerializer
import boto3
import json

role = get_execution_role()

# S3 prefixes
common_prefix = "hdts-sagemaker-testing"
batch_inference_input_prefix = "synthdata_tabular/input"
batch_inference_output_prefix = "synthdata_tabular/output"

## 3. Create the session

The session remembers our connection parameters to Amazon SageMaker. We'll use it to perform all of our Amazon SageMaker operations.

In [2]:
sagemaker_session = sage.Session()

## 4. Create Model

Now we use the above Model Package to create a model

In [5]:
model_package_arn = 'arn:aws:sagemaker:us-east-1:822940408628:model-package/synthdata-tabular'

In [19]:
model_name = 'synthdata'

content_type = 'application/zip'

#real_time_inference_instance_type = 'ml.t2.medium' does not support realtime inference
batch_transform_inference_instance_type = 'ml.m5.large'

In [6]:
def predict_wrapper(endpoint, session):
    return sage.predictor.Predictor(endpoint, session, content_type)

# Create a deployable model from the model package
model = ModelPackage(role=role,
                    model_package_arn=model_package_arn,
                    sagemaker_session=sagemaker_session,
                    predictor_cls = predict_wrapper)

## 5. Create-an-endpoint-and-perform-real-time-inference (not supported)

#### A. Create an endpoint

In [7]:
# #Deploy the model
# predictor = model.deploy(1,real_time_inference_instance_type,endpoint_name = model_name)

Once endpoint has been created, you would be able to perform real-time inference.

#### B. Create input payload

In [8]:
# file_name = 'input/article.txt'

#### C. Perform real-time inference

In [13]:
# !aws sagemaker-runtime invoke-endpoint \
#     --endpoint-name $model_name \
#     --body fileb://$file_name \
#     --content-type $content_type \
#     --region $sagemaker_session.boto_region_name \
#     output.txt

#### D. Visualize output

In [14]:
# import csv
# import json

# data = []

# with open('output.txt', 'r') as f:
#     csv_reader = csv.reader(f)
#     # csv_reader = csv.DictReader(f)
#     for row in csv_reader:
#         data.append(row)

# # json_output = json.dumps(data, indent=1)
# # print(json_output)
# print(data)

#### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [16]:
# predictor.delete_endpoint(delete_endpoint_config = True)

## 6. Perform batch inference

In this section, you will perform batch inference using multiple input payloads together. If you are not familiar with batch transform, and want to learn more, see these links:
1. [How it works](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-batch-transform.html)
2. [How to run a batch transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-batch.html)

### A. Configure the input S3 bucket folder

In [17]:
transform_input_folder = 'input'
transform_input = sagemaker_session.upload_data(transform_input_folder,common_prefix,batch_inference_input_prefix)

### B. Deploy the model

In [20]:
transformer = model.transformer(1,batch_transform_inference_instance_type,output_path="s3://"+common_prefix+"/"+batch_inference_output_prefix+"/")
transformer.transform(transform_input,content_type=content_type)
transformer.wait()

INFO:sagemaker:Creating model with name: synthdata-tabular-2023-08-14-17-14-20-804
INFO:sagemaker:Creating transform job with name: synthdata-tabular-2023-08-14-17-14-21-726


.......................................Starting the inference server with 2 workers.
[2023-08-14 17:20:57 +0000] [9] [INFO] Starting gunicorn 21.2.0
[2023-08-14 17:20:57 +0000] [9] [INFO] Listening at: unix:/tmp/gunicorn.sock (9)
[2023-08-14 17:20:57 +0000] [9] [INFO] Using worker: sync
[2023-08-14 17:20:57 +0000] [12] [INFO] Booting worker with pid: 12
[2023-08-14 17:20:57 +0000] [13] [INFO] Booting worker with pid: 13
169.254.255.130 - - [14/Aug/2023:17:21:08 +0000] "GET /ping HTTP/1.1" 200 12 "-" "Go-http-client/1.1"
169.254.255.130 - - [14/Aug/2023:17:21:08 +0000] "GET /execution-parameters HTTP/1.1" 404 2 "-" "Go-http-client/1.1"
['config.txt', 'hospital.csv']
Invoked with 285 records
/usr/local/lib/python3.8/dist-packages/rdt/transformers/base.py:132: FutureWarning: Future versions of RDT will not support the 'model_missing_values' parameter. Please switch to using the 'missing_value_generation' parameter to select your strategy.
  warnings.warn(
/usr/local/lib/python3.8/dist-pac

### C. Download the file from output S3 bucket folder

In [21]:
import os
s3_conn = boto3.client("s3")
with open('output2.csv','wb') as f:
    s3_conn.download_fileobj(common_prefix,batch_inference_output_prefix+'/'+'test.zip.out',f)
    print("Output file loaded from bucket")

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


Output file loaded from bucket


### D. Visualize data

In [23]:
import csv
import json

data = []

with open('output2.csv', 'r') as f:
    #csv_reader = csv.reader(f)
    csv_reader = csv.DictReader(f)
    for row in csv_reader:
        data.append(row)

json_output = json.dumps(data, indent=1)
print(json_output)

[
 {
  "198": "490",
  "9": "23",
  "e": "a",
  "11": "2",
  "Y": "X",
  "3": "3",
  "gynecology": "gynecology",
  "S": "R",
  "A": "F",
  "17006": "1106",
  "8.0": "8.0",
  "Trauma": "Trauma",
  "Moderate": "Moderate",
  "2": "2",
  "61-70": "51-60",
  "5123": "5864",
  "51-60": "31-40"
 },
 {
  "198": "132",
  "9": "14",
  "e": "c",
  "11": "11",
  "Y": "X",
  "3": "4",
  "gynecology": "TB & Chest disease",
  "S": "Q",
  "A": "D",
  "17006": "56943",
  "8.0": "8.0",
  "Trauma": "Trauma",
  "Moderate": "Moderate",
  "2": "2",
  "61-70": "31-40",
  "5123": "4885",
  "51-60": "31-40"
 },
 {
  "198": "318503",
  "9": "4",
  "e": "e",
  "11": "7",
  "Y": "X",
  "3": "3",
  "gynecology": "radiotherapy",
  "S": "Q",
  "A": "E",
  "17006": "26297",
  "8.0": "2.0",
  "Trauma": "Emergency",
  "Moderate": "Moderate",
  "2": "2",
  "61-70": "61-70",
  "5123": "5096",
  "51-60": "21-30"
 },
 {
  "198": "240",
  "9": "23",
  "e": "d",
  "11": "6",
  "Y": "Y",
  "3": "2",
  "gynecology": "gynecolog

### 4. Clean-up

### A. Delete the model

In [24]:
model.delete_model()

INFO:sagemaker:Deleting model with name: synthdata-tabular-2023-08-14-17-14-20-804


#### B. Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.